#### Задание
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [132]:
import requests

In [133]:
from bs4 import BeautifulSoup
import pandas as pd

In [135]:
def find(req, KEYWORDS, time,title,link):
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_id = post.parent.attrs.get('id')
        if not post_id:
            continue
        post_id = int(post_id.split('_')[-1])
        prewie=post.find('div', class_='post__text')
        prewie_lower = prewie.text.lower()
        if any([desired in prewie_lower for desired in KEYWORDS]):
                title_element = post.find('a', class_='post__title_link')
                data=post.find('span', class_="post__time")
                time.append(data)
                title.append(title_element.text)
                link.append(title_element.attrs.get('href'))
                continue
        hubs = post.find_all('a', class_='hub-link')
        for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in KEYWORDS]):
                title_element = post.find('a', class_='post__title_link')
                data=post.find('span', class_="post__time")
                if(title_element.text in title):
                    continue
                else:
                    time.append(data)
                    title.append(title_element.text)
                    link.append(title_element.attrs.get('href'))
               # так как пост уже нам подошел - дальше нет смысла проверять хабы
                continue

In [139]:
URL='https://habr.com/ru/all'
KEYWORDS = ['python', 'парсинг']
time=[]
title=[]
link=[]
req = requests.get(URL)
find(req,KEYWORDS, time, title, link)
news = pd.DataFrame(list(zip(time, title, link)),
columns =['data', 'title', 'link'])
print(news)

                data                                              title  \
0  [сегодня в 09:07]  Ловля цен. Практическое руководство в море зак...   

                               link  
0  https://habr.com/ru/post/525540/  
